In [ ]:
# This notebook will be for gathering the data to use with the AI machine

In [ ]:
# Import the necessary libraries
%pip install ohmysportsfeedspy
from ohmysportsfeedspy import MySportsFeeds #Wrapper provided by my score provider: https://github.com/MySportsFeeds/mysportsfeeds-python
import numpy as np
import pandas as pd
import boto3
from boto3.dynamodb.conditions import Attr
from boto3.dynamodb.conditions import Key
from datetime import datetime
from pytz import timezone

In [ ]:
#Setup connection to MySportsFeeds (abbreviated MSF going forward), loop through to get each week's data and then format it.
#TODO: Leverage parameter store to hold the key.

msf_key = input('MSF Key to Use: ') #Since I'll be storing this publicly I made this an input... you can register for your own key at mysportsfeeds.com.

msf = MySportsFeeds('2.1',verbose=False)
msf.authenticate(msf_key, 'MYSPORTSFEEDS')

In [ ]:
#Gather the basic game data and score for all of the games.

week_to_process = 3 #I am starting with week 3, because the games before week three won't have any meaningful data on previous games in the season.
while week_to_process <= 16:
    #Get the JSON data for the week.
    msf_games = msf.msf_get_data(league='nfl',season=('2021-regular'),feed='weekly_games',format='json',week=str(week_to_process))
    
    #Flatten the JSON data and create a data frame.
    week_data = pd.json_normalize(msf_games, record_path = ['games'])
    
    #Add the week number to each game.  It will be necessary because it is a sort key in my dynamoDB table.
    week_nums = []
    for index in week_data.index:
        week_nums.append(week_to_process)
    week_data['week'] = week_nums
    
    #Add the current week's data to the data_frame.
    if week_to_process == 3:
        game_data = week_data
    else:
        dfs = [game_data, week_data]
        game_data = pd.concat(dfs)
    
    week_to_process = week_to_process + 1
    

In [ ]:
#Format the data to have good names and be just the fields that I need.
game_data = game_data.filter(['schedule.id', 'schedule.awayTeam.id', 'schedule.homeTeam.id', 'schedule.venueAllegiance', 'score.awayScoreTotal','score.homeScoreTotal','week',
    'schedule.originalStartTime','schedule.startTime'])

game_data.rename(columns={"schedule.id": "game_id", 'schedule.awayTeam.id': 'awayTeam_ID', 'schedule.homeTeam.id': 'homeTeam_ID',
    'schedule.venueAllegiance': 'venueAliegiance', 'score.awayScoreTotal': 'awayScore', 'score.homeScoreTotal': 'homeScore',
    'schedule.originalStartTime': 'originalStartTime', 'schedule.startTime': 'startTime'}, inplace=True)


        

In [ ]:
#Add the line and the over_under.  Note that this data is coming from the actuals that I have stored in the database used by the actual game.
over_unders = []
home_lines = []
for index in game_data.index:
    the_game_id = int(game_data['game_id'].values[index])
    the_week = int(game_data['week'].values[index])
    dynamodb = boto3.resource('dynamodb')
    games_table = dynamodb.Table('LTHOIprod_games')
    lines = games_table.get_item(Key={'Week': the_week, 'Game_Id': the_game_id})
    lines = lines['Item']
    over_under_line = lines['Over_Under']
    home_line = lines['Home_Line']
    over_unders.append(float(over_under_line))
    home_lines.append(float(home_line))

game_data['over_under'] = over_unders
game_data['home_line'] = home_lines

In [ ]:
#Add the average points scored and average points against
import time
index = 0
home_team_average_for = []
home_team_average_against = []
away_team_average_for = []
away_team_average_against = []

for index in game_data.index:
    #Home Team Averages
    count = 0
    total_for = 0
    average_for = 0
    total_against = 0
    average_against = 0
    week_to_process = int(game_data['week'].values[index])
    msf_games = msf.msf_get_data(league='nfl',season='2021-regular',feed='seasonal_games',format='json',team=game_data['homeTeam_ID'].values[index])
    msf_games = msf_games['games']
    for game in msf_games:
        if game['schedule']['week'] < week_to_process:
            count = count + 1
            if game['schedule']['homeTeam']['id'] == game_data['homeTeam_ID'].values[index]:
                total_for = total_for + game['score']['homeScoreTotal']
                total_against = total_against + game['score']['awayScoreTotal']
            else:
                total_for = total_for + game['score']['awayScoreTotal']
                total_against = total_against + game['score']['homeScoreTotal']
                
    
    home_team_average_for.append((total_for / count))
    home_team_average_against.append((total_against / count))


    #Away Team Averages
    count = 0
    wins = 0
    losses = 0
    total_for = 0
    average_for = 0
    total_against = 0
    average_against = 0
    msf_games = msf.msf_get_data(league='nfl',season='2021-regular',feed='seasonal_games',format='json',team=game_data['awayTeam_ID'].values[index])
    msf_games = msf_games['games']
    for game in msf_games:
        if game['schedule']['week'] < week_to_process:
            count = count + 1
            if int(game['schedule']['homeTeam']['id']) == int(game_data['awayTeam_ID'].values[index]):
                total_for = total_for + game['score']['homeScoreTotal']
                total_against = total_against + game['score']['awayScoreTotal']
            else:
                total_for = total_for + game['score']['awayScoreTotal']
                total_against = total_against + game['score']['homeScoreTotal']
    
    away_team_average_for.append((total_for / count))
    away_team_average_against.append((total_against / count))
    
    #Pause 5 seconds so we don't exceed the rate limit on the API
    time.sleep(5)


game_data['home_team_average_for'] = home_team_average_for
game_data['home_team_average_against'] = home_team_average_against
game_data['away_team_average_for'] = away_team_average_for
game_data['away_team_average_against'] = away_team_average_against

In [ ]:
#Add the number of people betting on each side
the_over_bettors = []
the_under_bettors = []
the_home_bettors = []
the_away_bettors = []

for index in game_data.index:
    the_game_id = int(game_data['game_id'].values[index])
    dynamodb = boto3.resource('dynamodb')
    bets_table = dynamodb.Table('LTHOIprod_bets')
    all_bets = bets_table.query(KeyConditionExpression=Key('LS_Id').eq('d0ab7687-68eb-4753-a595-3a8acc50f3c0'), FilterExpression=Attr('Game_Id').eq(the_game_id) & Attr('Parent_Bet_Id').not_exists())
    home_bettors = 0
    away_bettors = 0
    over_bettors = 0
    under_bettors = 0
    for bet in all_bets['Items']:
        if ('Line_Bet' in bet and bet['Line_Bet'] == 1):
            home_bettors = home_bettors + 1
        elif ('Line_Bet' in bet and bet['Line_Bet'] == -1):
            away_bettors = away_bettors + 1
        elif (bet['Over_Under_Bet'] == 1):
            over_bettors = over_bettors + 1
        elif (bet['Over_Under_Bet'] == -1):
            under_bettors = under_bettors +1

    the_over_bettors.append(over_bettors)
    the_under_bettors.append(under_bettors)
    the_home_bettors.append(home_bettors)
    the_away_bettors.append(away_bettors)
    
game_data['over_bets'] = the_over_bettors
game_data['under_bets'] = the_under_bettors
game_data['home_bets'] = the_home_bettors
game_data['away_bets'] = the_away_bettors

In [ ]:
import pytz

#Find what the actual final line was.
the_final_line = []
the_final_ou = []
start_time = datetime.now()
tz_import = pytz.timezone('Etc/Zulu')
tz_target = pytz.timezone('US/Eastern')


for index in game_data.index:
    the_game_id = int(game_data['game_id'].values[index])
    start_time.replace(tzinfo=tz_import)

    #Convert Start time to Date Object and Set Date String for MSF Query
    start_time = datetime.strptime(game_data['startTime'].values[index], '%Y-%m-%dT%H:%M:%S.000Z').replace(tzinfo=tz_import)
    start_time = start_time.astimezone(tz_target)
    url_start_time = '' + start_time.strftime("%Y") + start_time.strftime("%m") + start_time.strftime("%d")                                                                                             

    #If there is an originalStartTime then over-ride the MSF Query String
    if game_data['originalStartTime'].values[index] is not None:
        original_start_time = datetime.strptime(game_data['originalStartTime'].values[index], '%Y-%m-%dT%H:%M:%S.000Z').replace(tzinfo=tz_import)
        original_start_time = original_start_time.astimezone(tz_target)
        url_start_time = '' + original_start_time.strftime("%Y") + start_time.strftime("%m") + start_time.strftime("%d")
        print('{} used {} instead.'.format(the_game_id, url_start_time))

    msf_lines = msf.msf_get_data(league='nfl',season='2021-regular',feed='daily_game_lines',format='json',provider='888Sport',game=the_game_id,date=url_start_time)
    msf_spreads = msf_lines['gameLines'][0]['lines'][0]['pointSpreads']
    msf_ous = msf_lines['gameLines'][0]['lines'][0]['overUnders']

    closest_to_kickoff = datetime(1970, 1, 1).replace(tzinfo=tz_import)
    line = 0
    for spread in msf_spreads:
        current_time = datetime.strptime(spread['asOfTime'], '%Y-%m-%dT%H:%M:%S.000Z').replace(tzinfo=tz_import)
        current_time = current_time.astimezone(tz_target)
        if (current_time > closest_to_kickoff) and (current_time < start_time) and (spread['pointSpread']['gameSegment'] == 'FULL'):
            closest_to_kickoff = current_time
            line = spread['pointSpread']['homeSpread']

    closest_to_kickoff = datetime(1970, 1, 1).replace(tzinfo=tz_import)
    ou_final = 0
    for ou in msf_ous:
        current_time = datetime.strptime(ou['asOfTime'], '%Y-%m-%dT%H:%M:%S.000Z').replace(tzinfo=tz_import)
        current_time = current_time.astimezone(tz_target)
        if (current_time > closest_to_kickoff) and (current_time < start_time) and (ou['overUnder']['gameSegment'] == 'FULL'):
            closest_to_kickoff = current_time
            ou_final = ou['overUnder']['overUnder']

    the_final_line.append(line)
    the_final_ou.append(ou_final)

    time.sleep(15)

game_data['final_line'] = the_final_line
game_data['final_over_under'] = the_final_ou


In [ ]:
#Store the data to CSV
bucketname = "burgherjon-football-data"
key = "input_data/gathered_data.csv"

game_data.to_csv('gathered_data.csv')

s3 = boto3.resource('s3')
s3.meta.client.upload_file('gathered_data.csv', bucketname, key)


In [ ]:
game_data.head()

In [ ]:
game_data.info()

game_data.describe()

In [ ]:
game_data.describe()